
Date: 2/09/2018



Environment: Python 3.6.1 and Jupyter notebook

Libraries used: Main libraries used for assignment:
* re (for regular expression, included in Anaconda Python 3.6) 
* json (for writing json converted files, included in Anaconda Python 3.6) 
* sys (for finding the system version, included in Anaconda Python 3.6) 


## 1) Introduction


This task explores the steps to extract structured data from an unstructured data source. Using regex pattern, data is extracted from files and read and saved in json and xml files. 

Logic:
1) A regex pattern is created to identify all keys present in resumes   
2) Using python indexing, all data between two keys is stored for a key as value.   
3) final output is sent to json and xml files


## 2) Importing Libraries


In [75]:
import sys
import re

print (sys.version_info)

sys.version_info(major=3, minor=6, micro=1, releaselevel='final', serial=0)


## 3) Reading the resume files assigned to me 

#### While data is read, it is also cleaned in order to save cleaned data for further processing 

In [76]:
with open('29389690.dat','r') as input_file: 
    task1_ipfile=input_file.read() #---------------------1
    task1_ipfile= re.sub('to:', 'to', task1_ipfile) #---2
    task1_ipfile= re.sub('&', 'and', task1_ipfile) #----3
    task1_ipfile= re.sub('<', 'less than', task1_ipfile) #-----4
    task1_ipfile= re.sub('>', 'greater than', task1_ipfile) #---------5
    task1_ipfile=(re.sub(r'(?:\n|^|_)([A-Za-z]+[\s_]?[A-Za-z]+)[/]', r'\n\1: NA', task1_ipfile)) #--------6
    resume = re.split("----+\n", task1_ipfile)#--------7
input_file.close()

#Above lines are explained below:
#1 - File reading
#2 - converting whenever word=to: --> word=to,this is required to fetch the data in correct format
#3 - converting whenever word=& --> word=and,this is required to fetch the data in correct format
#4 - converting whenever word=< --> word=less than,this is required to fetch the data in correct format
#5 - converting whenever word=> --> word=greater than,this is required to fetch the data in correct format
#6 - A regex pattern to replace whenever blank value is present against any word to "NA"
    #Explanation of regex pattern (?:\n|^|_)([A-Za-z]+[\s_]?[A-Za-z]+)[/]
        #(?:\n|^|_) --> identiifes a non capturing group, any new line or start of string or starting with underscore
        #([A-Za-z]+[\s_]?[A-Za-z]+) --> A group which matches capital/lower word followed by space/underscore and capital/lower word
        #[:/] --> the ending character with / or colon
        #when above pattern is matched, it is substituted to NA using re.sub command
#7 - each resume is split based on lines --------


#Further cleaning of data
resume = [x for x in resume if x != ''] #if any blank resume is present, it is discarded
print("Total resume data processed=",len(resume))

Total resume data processed= 32069


### 4) Defining Function to perform set of actions

#### 4-1) divide_sub_pattern function is created to divide sentences inside each file whenever semicolon is met. This is required to store  in json and xml format

In [77]:
#When semicolon is met, sentences are divided. This is required for creating sub tags for json and xml files.
def divide_sub_pattern(data):
    data=data.split(';')
    return data

#### 4-2) create_sub_element function is required to create a sub-key for each matched key

In [78]:
#For qualification, procedure, description and responsibility we need to divide the values into sub division,
#below function will perform, it divides the key into sub-key and further assigns the value to it.
def create_sub_element(key,value):
    key=re.sub('\s+', '_',key.lower())
    if "qual" in key:
        data1={"qualification":value}
    elif "proc" in key:
        data1={"procedure":value}
    elif "desc" in key :
        data1={"description":value}
    elif "resp" in key:
        data1={"responsibility":value}
    else: 
        data1=key
    return data1

#### 4-3) A function to extracts first, each resume data in dictionary format and pre-process the data in desired format.

In [79]:
def extract_key_value(resume_keys,resume_index):
    key=[] #----------------------------------------------list which will store keys.
    value=[] #list which will store values with respect to keys.    
    for item in range(len(resume_keys)): # to iterate over keys to fetch respective values
        name_of_key = resume_keys[item] #stores the name of key passed
        
#Logic is to find starting and ending position of all keys and then fetch value between two keys. 

        end_char=(re.search(resume_keys[item], resume_index).end()+1) #ending position of a key
        if item+1 <len(resume_keys): 
            start_char=(re.search(resume_keys[item+1], resume_index).start()-1) #starting position of next consecutive key
            value_of_key= resume_index [end_char:start_char] # fetches value between 1st and 2nd key; whcih is output of 1st key
            
            #data cleaning steps
            value_of_key=value_of_key.replace('\n',' ') #replaces backslash with space
            value_of_key = re.sub('\s+', ' ', value_of_key) #replaces multiple spaces with single space.
            
            #Next step is to create child key-value pair, only if values contains multiple lines.
            if " - " in value_of_key : #if any value starts with multiple steps, perform below actions
                value_of_key=value_of_key.replace(' - ','') # replace hypen with space
                value_of_key=divide_sub_pattern(str(value_of_key)) # calls fucntions which splits strings by ; 
                value_of_key=create_sub_element(name_of_key,value_of_key) #creation of child-name and value
            else :
                value_of_key=value_of_key  
                value_of_key=value_of_key.lstrip() #removes leading spaces if any
                
        #This part is to extract only for last key for each resume.
        else:
            start_char=(re.search(resume_keys[item], resume_index).start()-1)
            value_of_key= resume_index [end_char:len(resume_index)] #ending position of last key till end of file.
            
            #rest of steps are same of above
            value_of_key=value_of_key.replace('\n',' ')
            value_of_key = re.sub('\s+', ' ', value_of_key)
            if " - " in value_of_key :
                value_of_key=value_of_key.replace(' - ','')
                value_of_key=divide_sub_pattern(str(value_of_key))
                value_of_key=create_sub_element(name_of_key,value_of_key)
            else :
                value_of_key=value_of_key
                value_of_key=value_of_key.lstrip()
        key.append(re.sub('\s+', '_',name_of_key.lower()))  # replace underscore b/w two words if missing and capitalize
        value.append(value_of_key)
    new_dict= dict(zip(key,value)) #zipping all keys and values to form a dicitonary
    return new_dict

### 5) Creates new list, which will contain cleaned resumes.

Logic: identify keys by a regex pattern and calls function(extract_key_value) to extract all values for respective keys.

In [80]:
resume_dict=[]
for item in range(len(resume)):
    pat_key=r'(?:\n|^|_)([A-Za-z]+[\s_]?[A-Za-z]+)[:/]'
    pat=re.findall(pat_key,resume[item]) #store each regex matched and output will be keys for each resumes.
    resume_dict.append(extract_key_value(pat,resume[item])) #resume_dict stores all resumes in list format.
    
# regex explanation:(?:\n|^|_)([A-Za-z]+[\s_]?[A-Za-z]+)[:/]
    #(?:\n|^|_) --> identiifes a non capturing group, any new line or start of string or starting with underscore
    #([A-Za-z]+[\s_]?[A-Za-z]+) --> A group which matches capital/lower word followed by space/underscore and capital/lower word
    #[:/] --> the ending character with / or colon

------------------------
### 6) Writing output to files 

#### 6-1) Output to json file

In [81]:
import json
with open('29389690.json', 'w') as f:
    listing={} #first creare a root node 
    listing['listing']=resume_dict # add a new key to dictionary. 
    f.write(json.dumps({'listings': (listings)},indent=4)+"\n\n\n\n\n") # add final output
f.close()

#### 6-2) Ouput to xml file

##### Creating a function to write a xml

In [82]:
#recursive function to create parent and child tags
'''
l= if value in key value pair is of type string than it will create tags and hold its value 
l2= if value in key value pair is of type list than it will create child tags 
l3= holds closing tag name if value in key, value pair is dictionary type
lr= final output string contaings all resume data in xml format
'''
def file2xml(D, l="",l2="",l3="",lr=''): #
    if type(D) == dict: 
        for key, value in D.items():
            lr=lr
            if isinstance(value, dict):
                l3="</"+key+">"
                lr+="<"+key+">"+file2xml(value)+l3                             
            elif isinstance(value, tuple) or isinstance(value, list):
                for i in range(len(value)):
                    val=''.join(value[i])
                    l2="<"+key+">"+val+"</"+key+">"
                    lr+=l2
                lr+=l3
            else:
                l="<"+key+">"+str(value)+"</"+key+">"
                lr+=l    
    else :
        l+=str(D)+l3
        lr+=l
    return lr



xml_file=[]  #create blank list to store final output for xml
# for loop to iterate over each resume and convert it into xml format
for item in range(len(resume)):    
        xml_file.append("<listing>"+file2xml(resume_dict[item])+"</listing>") 
        # make listing as a parent nood for each resume 

In [83]:
#Writing final output to xml file

with open('29389690.xml', 'w') as f:
        f.write("<listings>"+''.join(xml_file)+"</listings>")
f.close()